# Detailed player analysis

In [ ]:
%reload_ext autoreload
%autoreload 2
import os 
import sys
import pandas as pd
import numpy as np
from datetime import datetime

import plotly 
import plotly.graph_objects as go
import time

sys.path.insert(0, "./../../src/")
from data_loader import load_data, load_player_data, player_data_preprocessing

player_df = pd.read_csv("player_info_v3.csv")
columns = ["Player ID"] + list(player_df.columns)[1:]
player_df.columns = columns

player_df.head()

# Distribution of stays in the NBA

In [ ]:
# player_df["season_name_int"] = player_df["season_name"].str.replace("-", "").astype(int)
grouped_player = player_df[["season_name", "Player ID"]].groupby("Player ID")\
                    .count().sort_values(by="season_name", ascending=False)
grouped_player.columns = ["season_count"]

print(f"Average stay in NBA is: {grouped_player['season_count'].mean()}")

fig_hw = go.Figure([go.Histogram(x=grouped_player["season_count"])])
fig_hw.update_layout(
    title_text="Distribution of player stays in NBA", 
    xaxis_title="Seasons in NBA",
    yaxis_title="Count"
)
fig_hw.show()

In [ ]:
top_10_longest_in_nba = grouped_player.reset_index()\
                        .merge(player_df, left_on='Player ID', right_on='Player ID')\
                        [["player_name", "season_count"]].drop_duplicates()[:13]
fig_countries = go.Figure([go.Bar(
    x=top_10_longest_in_nba["player_name"], 
    y=top_10_longest_in_nba["season_count"]
    )]
)
fig_countries.update_layout(
    title_text="Top 13 players with the most seasons played (2000-2019)", 
    yaxis_title="Count",
    xaxis_title="Player",
)
fig_countries.update_traces(
    textposition='outside'
)
fig_countries.show()

display(top_10_longest_in_nba)

In [ ]:
list(top_10_longest_in_nba["player_name"])